In [34]:
import numpy as np
import cv2
import time

### Testing OpenCV built-in trackers (currently legacy) in video:

In [44]:
def Track_ROI_video(video, tracker_name,  dims = (640, 480), show_fps=False):
    
    # Create MultiTracker object
    multi_tracker = cv2.legacy.MultiTracker_create()
    
    # Tracker Dict:
    tracker_dict = {'CSRT': cv2.legacy.TrackerCSRT_create,
                    'KCF': cv2.legacy.TrackerKCF_create,
                    'BOOSTING': cv2.legacy.TrackerBoosting_create,
                    'MIL': cv2.legacy.TrackerMIL_create,
                    'TLD': cv2.legacy.TrackerTLD_create,
                    'MEDIANFLOW': cv2.legacy.TrackerMedianFlow_create,
                    'MOSSE': cv2.legacy.TrackerMOSSE_create}
    
    # ROI State Variable
    TRACKING_ON = False

    # Select Desired Video
    video = cv2.VideoCapture(video)
    video_fps = video.get(cv2.CAP_PROP_FPS)
    
    # record fps during tracking:
    fps_record = []

    # Video stream loop:

    while True:
        ret, frame = video.read()

        start = time.time()

        if not ret: # Break the loop if no video is load.
            break

        frame = cv2.resize(frame,(dims))
        
        if (TRACKING_ON == False):
            cv2.putText(frame, "Ready to Track, press P to select the Region of Interest.", (100, 510),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 80, 80), 1)

        #**************************************************************************************************************************
        #**********************************************/Selected ROI Tracker/******************************************************

        # Press P to Select the Region of Interest and Track it using CSRT Tracker.
        # Press K after a tracking start to stop it.
        # Press Q to break the video stream and close windows.

        key = cv2.waitKey(1) or 0xff

        #************************************* PRESS Q TO CLOSE STREAM ************************************************************
        #**************************************************************************************************************************
        if key == ord('k'):
            break
        #**************************************************************************************************************************

        #************************************* PRESS P TO SELECT TRACKING OBJECT **************************************************
        #**************************************************************************************************************************
        if key == ord('p'):

            TRACKING_ON = True
            r = cv2.selectROI("frame", frame)

            # Add Tracker
            multi_tracker.add(tracker_dict[tracker_name](), frame, r)

        #**************************************************************************************************************************
        #**************************************************************************************************************************

        if (TRACKING_ON):

            # get updated location of objects in subsequent frames
            success, boxes = multi_tracker.update(frame)

            # draw tracked objects
            for i, newbox in enumerate(boxes):
                p1 = (int(newbox[0]), int(newbox[1]))
                p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
                cv2.rectangle(frame, p1, p2, (0, 0, 255), 2, 1)
                cv2.putText(frame, "Obj: {}".format(i+1), (p1[0], p1[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
                cX = int(p1[0] + (p2[0]-p1[0])/2)
                cY = int(p1[1] + (p2[1]-p1[1])/2)
                cv2.circle(frame, (cX, cY), 3, (0, 0, 255), -1)
                cv2.putText(frame, tracker_name, (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)

        #**************************************************************************************************************************

        #************************************* PRESS R TO RESET CURRENTLY TRACKING ************************************************
        #**************************************************************************************************************************
        if key == ord('r'):
            TRACKING_ON = False
            multi_tracker.clear()
            multi_tracker = cv2.legacy.MultiTracker_create() # clear tracking object
            
        #**************************************************************************************************************************
        #************************************* SHOW VIDEO STREAM ******************************************************************

        end = time.time()
        frame_time = (end - start) + 0.0001
        fps = np.floor(1/frame_time)
        
        if (fps > video_fps):
            time.sleep(1/video_fps)
            fps = video_fps
        
        if (show_fps):
            cv2.putText(frame, "FPS: {}".format(fps), (0, 430),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 50, 255), 2)
            
        if TRACKING_ON:
            fps_record.append(fps)

        cv2.imshow("frame", frame)
        #**************************************************************************************************************************    

    video.release()
    cv2.destroyAllWindows()
    
    return fps_record

In [43]:
# Starting Setup:
video = 'people.mp4'

# Tracker List: 'CSRT', 'KCF', 'BOOSTING', 'MIL', 'TLD', 'MEDIANFLOW', 'MOSSE'

# Desired tracker:
tracker_name = 'BOOSTING'

fps_record = Track_ROI_video(video, tracker_name, dims = (640, 480), show_fps=True)
print('FPS mean during tracking:', np.mean(fps_record))

FPS mean during tracking: 16.43031784841076
